## Name: Gopani Kenil G. 
## Roll No: CE049 
## ID: 19CEUOS080
##  Lab-7

In [22]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [23]:
# Setting Up Hyperparameters and Data Set Parameters

num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 51

In [24]:
# Loading and Preparing the MNIST Data Set

from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [25]:
# Shuffling and Batching the Data

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [26]:
# Initializing Weights and Biases

W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [27]:
# Defining Logistic Regression and Cost Function

def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [28]:
# Defining Optimizers and Accuracy Metrics

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

In [29]:
# Optimization Process and Updating Weights and Biases

def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [30]:
# The Training Loop

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 51, loss: 538.663635, accuracy: 0.769531
step: 102, loss: 102.817978, accuracy: 0.890625
step: 153, loss: 224.749939, accuracy: 0.828125
step: 204, loss: 74.882690, accuracy: 0.921875
step: 255, loss: 91.350616, accuracy: 0.910156
step: 306, loss: 138.076248, accuracy: 0.859375
step: 357, loss: 66.994141, accuracy: 0.933594
step: 408, loss: 614.191833, accuracy: 0.851562
step: 459, loss: 56.102993, accuracy: 0.933594
step: 510, loss: 90.010178, accuracy: 0.902344
step: 561, loss: 88.799721, accuracy: 0.917969
step: 612, loss: 110.771179, accuracy: 0.910156
step: 663, loss: 70.352547, accuracy: 0.929688
step: 714, loss: 81.328079, accuracy: 0.910156
step: 765, loss: 90.933945, accuracy: 0.925781
step: 816, loss: 71.745239, accuracy: 0.921875
step: 867, loss: 104.933945, accuracy: 0.917969
step: 918, loss: 44.586540, accuracy: 0.964844
step: 969, loss: 50.028866, accuracy: 0.937500


In [31]:
# Testing Model Accuracy Using the Test Data

pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.912400
